In [ ]:
import pandas as pd
import os

# 1. Setup file paths
folderpath = "data"

# This list comprehension now looks "recursively" through the subfolders
filepaths = []
for root, dirs, files in os.walk(folderpath):
    for file in files:
        # Only grab CSVs and ignore hidden Mac system files (starting with ._)
        if file.endswith('.csv') and not file.startswith('._'):
            filepaths.append(os.path.join(root, file))

print(f"Found {len(filepaths)} CSV files.")

# 2. Efficiently join data using a generator
if filepaths:
    print("Starting the merge... this may take a minute due to file size.")
    df = pd.concat((pd.read_csv(f, low_memory=False) for f in filepaths), ignore_index=True)
    print("Successfully merged all files!")
    print(f"Final Dataframe Shape: {df.shape}")
else:
    print("No CSV files found. Double-check that the unzipped CSVs are inside the folders.")

df.head()

In [8]:
import requests
import json
import pandas as pd
from datetime import datetime
import time

Token = 'UeqLvUxbtdOBQCjaOrsSObCYkeYoDbYT'

# Station GHCND:USW00014732 is LaGuardia Airport
url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31'

r = requests.get(url, headers={'token':Token})
print(f"Status Code: {r.status_code}")

if r.status_code == 200:
    d = json.loads(r.text)
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    
    # Convert from tenths of Celsius to Celsius
    temps = [float(item['value'])/10.0 for item in avg_temps]
    dates_temp = [item['date'] for item in avg_temps]

    df_temp = pd.DataFrame({'date': dates_temp, 'avgTemp': temps})
    # Clean the date format to match bike data
    df_temp['date'] = pd.to_datetime(df_temp['date']).dt.date
    
    # Export as required
    df_temp.to_csv('nyc_weather_2022.csv', index=False)
    print("Weather data successfully saved to 'nyc_weather_2022.csv'")
else:
    print(f"Failed. Status: {r.status_code}. Response: {r.text}")

Status Code: 200
Weather data successfully saved to 'nyc_weather_2022.csv'


In [9]:
# 1. Prepare the bike data date column
# This handles the 'started_at' column which is a timestamp
df['date'] = pd.to_datetime(df['started_at']).dt.date

# 2. Perform the merge
# We use a 'left' join to keep all 30M+ bike trips and attach weather where it matches
df_merged = df.merge(df_temp, on='date', how='left')

# 3. Final Export (The big one!)
# This file will be very large.
df_merged.to_csv('merged_citibike_2022.csv', index=False)

print(f"Final Merge Complete! Dataset shape: {df_merged.shape}")
df_merged.head()

Final Merge Complete! Dataset shape: (29838806, 15)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp
0,0CC1D7F53FA21F9A,electric_bike,2022-11-02 05:54:11.481,2022-11-02 06:08:27.471,Park Ave & E 162 St,8016.07,Jerome Ave & W 193 St,8619.02,40.825701,-73.915644,40.866590,-73.897940,casual,2022-11-02,17.6
1,EDAECDAE6BF903DE,classic_bike,2022-11-02 18:20:15.611,2022-11-02 19:00:24.787,Broadway & W 61 St,7014.12,Leonard St & Church St,5359.11,40.770030,-73.981968,40.717571,-74.005549,casual,2022-11-02,17.6
2,37C06FED49069B80,electric_bike,2022-11-04 18:39:39.873,2022-11-04 18:44:01.851,W 54 St & 11 Ave,6955.05,11 Ave & W 59 St,7059.01,40.768333,-73.992573,40.771497,-73.990460,member,2022-11-04,15.2
3,63751973E9A95FB1,classic_bike,2022-11-09 18:02:29.616,2022-11-09 18:19:28.693,Broadway & W 41 St,6560.01,11 Ave & W 59 St,7059.01,40.755136,-73.986580,40.771497,-73.990460,member,2022-11-09,8.7
4,F7410DEDF925FBA8,electric_bike,2022-11-12 10:23:11.805,2022-11-12 10:28:24.794,William St & Pine St,5065.12,Leonard St & Church St,5359.11,40.707317,-74.008854,40.717571,-74.005549,member,2022-11-12,19.7
